In [1]:
import math
import numpy as np
import pandas as pd
import pints
import matplotlib.pyplot as plt
import os
plt.rcParams.update({'font.size': 24})
from Newton_model import wrappedNewton, newtonRaphsonFT

ModuleNotFoundError: No module named 'Newton_model'

In [2]:
# setting up models
measurements = 45496 # 47564
trial = newtonRaphsonFT(startPotential= -0.15, revPotential = -0.75, rateOfPotentialChange = -22.35e-3, numberOfMeasurements = measurements, theta_X_Initial = 1.0, theta_Z_Initial = 0.0, kappa0_1 = 3400.0, kappa0_2 = 3400.0 )

data_name = 'data_90992'

model = wrappedNewton(numberOfMeasurements = measurements, initaldiscard = 0.025, enddiscard = 0.875)
file_name = data_name + '.txt'
data_file = os.path.join(os.path.join('Data', 'processed'), file_name)
experimental_data = pd.read_csv(data_file, sep='\t')

exp_times = experimental_data.times
exp_current = experimental_data.current
exp_current_dimless = exp_current/trial.I0

exp_FT_data= model.FT_and_reduce_to_harmonics_4_to_12(exp_current_dimless)
exp_freq = model.frequencies_for_harmonics_4_to_12(exp_times)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/processed/data_90992.txt'

In [ ]:
class ComplexRootMeanSquaredError(pints.ProblemErrorMeasure):
    def __init__(self, problem):
        super(ComplexRootMeanSquaredError, self).__init__(problem)

        if not isinstance(problem, pints.SingleOutputProblem):
            raise ValueError(
                'This measure is only defined for single output problems.')
        
        self.measurements = 45496
        self.initaldiscard = int(0.025*self.measurements )
        self.enddiscard = int(0.875*self.measurements )
        FT = np.fft.fft(self._values)
        sp = FT[:self.measurements]
        self.FT_values = sp[self.initaldiscard:self.measurements -  self.enddiscard]
        # np.absolute takes the element wise absoulte value of real numbers, 
        # and the element-wise modules/eulcdiean norm/ absoulte value of comlex numbers
        # i.e for: z = a +bi
        # |z| = sqrt(a^2+b^2)
    def __call__(self, x):

        simulation = self._problem.evaluate(x)
        FT_sim = np.fft.fft(simulation)
        halved = FT_sim[:self.measurements]
        reduced_FT_sim = halved[self.initaldiscard:self.measurements -  self.enddiscard]


        complex_diff = reduced_FT_sim - self.FT_values

        magnatuide = np.absolute(complex_diff)

        squares = np.square(magnatuide)

        sumed = np.sum(squares)

        return np.sqrt(sumed)

In [ ]:

problem = pints.SingleOutputProblem(model, exp_times, exp_current_dimless)

score = ComplexRootMeanSquaredError(problem)

# boundary conditions
# kappa0_1, kappa0_2, epsilon0_1, epsilon0_2, mew, zeta, sigma
#[3400.0, 3400.0, -0.437459534627, -0.46045114238, -0.031244092599793216, 1.0]
e_min = model.startPotential + 0.2*(model.revPotential - model.startPotential)
e_max = model.startPotential + 0.8*(model.revPotential - model.startPotential)
# f.write("e_min: %e\r\n" % e_min)
# f.write("e_max: %e\r\n" % e_max)
# print('e_min: ',e_min)
# print('e_max: ',e_max)
lower_bounds = np.asarray([0.0, 0.0, e_max, e_max, -0.314, 0.0])#, 0.1])
upper_bounds = np.asarray([4000.0, 4000.0, e_min, e_min, 0.314, 10])#, 4.5])

boundaries = pints.RectangularBoundaries(lower_bounds, upper_bounds)
print("Score at papers solution:", score(real_parameters))
found_params = [5.06670907862494602e+00,  9.89015401127633936e+02,
                -2.99712963556115097e-01, -4.46238055110197540e-01, 
                 1.90280568779692127e-02,  1.31115829339899492e+00]
print("Score at found solution:", score(found_params))
# Score at papers solution: 877785.9197391798
# Score at found solution: 1006276.6135752603

In [ ]:
total_runs = 20
dims = real_parameters.shape
print('dims: ', dims)
print('dims[0]: ', dims[0])
params_matrix = np.zeros((total_runs, dims[0]))
for run in range(total_runs):
        
    if run > 0:
        f = open(os.path.join( folder, output_file_name),"a")

    f.write("\r\n\r\n" + 40*"*" + " Run: %d " % run + 40*"*" + "\r\n\r\n")
    print('\n\n' + 40*"*" + ' Run: ', run, ' ' + 40*"*" + '\n\n')
    accuracy = (2 + run)
    # f.write("Threshold for stopping: %e\r\n\r\n" % pow(10,-accuracy))
    # print('Threshold for stopping: ', pow(10,-accuracy))

    ranges = upper_bounds - lower_bounds 

    starting_points = np.copy(lower_bounds)
    for i in range(len(ranges)):
        starting_points[i] += ranges[i] * np.random.uniform(low = 0.001, high = 0.999)
    
    print('lower_bounds: ', lower_bounds)
    print('upper_bounds: ', upper_bounds)

    # if run == 11:
    #     starting_points = real_parameters
    # print('random starting points: ', starting_points)

    f.write("lower_bounds: ")
    for i in lower_bounds:
        f.write("%e, " % i)
    f.write("\r\nupper_bounds: ")
    for i in upper_bounds:
        f.write("%e, " % i)
    f.write("\r\nRandom starting_points: ")
    for i in starting_points:
        f.write("%e, " % i)
    f.write("\r\n")

    # transformation
    transform = pints.RectangularBoundariesTransformation(boundaries)

    # optimising boundaries=boundaries,
    opt = pints.OptimisationController(
        score,
        x0=starting_points,
        method=pints.CMAES,
        transform = transform)

        
    #opt.set_max_unchanged_iterations(iterations=50, threshold=pow(10,-accuracy))
    #opt.set_max_unchanged_iterations(iterations=100)
    opt.set_parallel(parallel=True)
    #opt.set_max_iterations(iterations=10)
    opt.set_log_interval(iters=20, warm_up=3)
    #opt.set_threshold(threshold=-??????)
    
    found_parameters, found_value =  opt.run()
        #CMAES, PSO, SNES, XNES
    # output results

    print('random starting points: ', starting_points)

    f.write("Found solution: ")
    for k, x in enumerate(found_parameters):
        f.write(pints.strfloat(x) +", ")

    print('         Found solution:          True parameters:' )
    name = 0
    for k, x in enumerate(found_parameters):
        print( pints.strfloat(x) + '    ' + pints.strfloat(real_parameters[k]) + '  :' + parameter_order[name])
        name = name + 1

    print('lower_bounds: ', lower_bounds)
    print('upper_bounds: ', upper_bounds)

    f.write("\r\n\r\nFound solution:          True parameters:\r\n")
    name = 0
    for k, x in enumerate(found_parameters):
        f.write(pints.strfloat(x) + '    ' + pints.strfloat(real_parameters[k])+ '  :' + parameter_order[name]+ '\r\n')
        name = name + 1

    params_matrix[run, :] = found_parameters

    # plotting current for found parameters over the experimentally data

    xaxis = exp_times #model.potentialRange
    xaxislabel = "time/s" # "potential/V"
    solution = model.simulate(found_parameters, exp_times)


    plt.figure(figsize=(18,10))
    plt.title("optimised and experimental values")
    plt.ylabel("Fourier transformed current/dimless")
    plt.xlabel(xaxislabel)
    plt.plot(xaxis, exp_current_dimless,'r', label='experiment_'+str(real_parameters[0])+'_'+str(real_parameters[1]))
    plt.plot(xaxis ,solution,'b', label='optimised_'+str(found_parameters[0])+'_'+str(found_parameters[1]))
    plt.legend(loc='best')
    plt.savefig(os.path.join( folder, 'experiment against optimised with potential run '+str(run)+'.png'))
    #plt.show()
    plt.close()

    plt.figure(figsize=(18,10))
    plt.title("optimised and experimental values")
    plt.ylabel("Fourier transformed current/dimless")
    plt.xlabel(xaxislabel)
    plt.plot(xaxis ,solution,'b', label='optimised_'+str(found_parameters[0])+'_'+str(found_parameters[1]))
    plt.plot(xaxis,exp_current_dimless,'r', label='experiment_'+str(real_parameters[0])+'_'+str(real_parameters[1]))
    plt.legend(loc='best')
    plt.savefig(os.path.join( folder, 'optimised against experiment with potential run '+str(run)+'.png'))
    #plt.show()
    plt.close()

    print("Score at true solution:", score(real_parameters))
    print("Score at found solution:", score(found_parameters))
    f.write("\r\nScore at true solution:  %.16e\r\n" % score(real_parameters))
    f.write("Score at found solution: %.16e\r\n" % score(found_parameters))

    run += run
f.close()